In [3]:
%reload_ext autoreload
%autoreload 2

import os, pickle
import numpy as np
import pandas as pd
import main_funcs as mfun
import utils_funcs as utils
import plot_funcs as pfun
import matplotlib.pyplot as plt
import pickle
from scipy import stats
import glob
from scipy.signal import resample

pfun.set_figure()

infoPath = 'C:\\Users\\Huriye\\Documents\\code\\clapfcstimulation\\analysis\\infoForAnalysis.pkl'
info = pd.read_pickle(infoPath) 

## Parameters
fRate = 1000/30
responsiveness_test_duration = 1000.0 #in ms 
simulationDur_ms = 350.0 # in ms
simulationDur = int(np.ceil(simulationDur_ms/fRate))
pre_frames    = 2000.0# in ms
pre_frames    = int(np.ceil(pre_frames/fRate))
post_frames   = 6000.0 # in ms
post_frames   = int(np.ceil(post_frames/fRate))
analysisWindowDur = 1500 # in ms
analysisWindowDur = int(np.ceil(analysisWindowDur/fRate))
shutterLength     = int(np.round(simulationDur_ms/fRate))
tTypes = ['All','Both', 'onlyOpto', 'onlyVis']

In [5]:
pathname[-21:-1]+'_DFF_mean_'+str(indC)

'\\2023-08-16_2306_001_DFF_mean_1'

In [8]:
info.figsPath+'\\cellDFF\\'

'C:\\Users\\Huriye\\Documents\\code\\clapfcstimulation\\figs\\cellDFF\\'

In [26]:
# Loop through all recordings
# For each recording one file saved, each index is a cell 
gh = 272
for ind, recordingDate in enumerate(info.recordingList.recordingDate):
   savepathname = info.recordingList.analysispathname[ind]
   filenamePAQ_analysis = [f for f in glob.glob(savepathname + '2extracted_variables.pkl')] # paq analysis file
   if  (info.recordingList.calciumImaging[ind] ==0) & (info.recordingList.pupilImaging[ind] ==0):
      info.recordingList.loc[ind,'extractedVar'] = 0
   elif (len(filenamePAQ_analysis)==1):
      info.recordingList.loc[ind,'extractedVar'] = 1 
   elif (info.recordingList.stimuliFamiliarity[ind] == 5):
      info.recordingList.loc[ind,'extractedVar'] = 0
   else:
      if ind==272: #try:
         print(str(ind) + ': Creating: ' + info.recordingList.analysispathname[ind])
         #Create a huge dictionary with all cells and parameters for each cell
         pathname = info.recordingList.analysispathname[ind]
         print('Creating the dict for:' + pathname)

          ########## Organise stimuli times 
         if  (info.recordingList.paqExtraction[ind] ==1):
            paqData = pd.read_pickle (pathname +'paq-data.pkl')
            paqRate = paqData['rate']
            # Get the stim start times 
            frame_clock    = utils.paq_data (paqData, 'prairieFrame', threshold_ttl=True, plot=False)
            optoStimTimes  = utils.paq_data (paqData, 'optoLoopback', threshold_ttl=True, plot=False)
            if   len(optoStimTimes)>500:
               print('Opto stim times are weird, check it out')
            else: # the frame_clock is slightly longer in paq as there are some up to a sec delay from
               # microscope to PAQI/O software.  
               optoStimTimes = utils.stim_start_frame (paq=paqData, stim_chan_name='optoLoopback',
                                                   frame_clock=None,stim_times=None, plane=0, n_planes=1)
               visStimTimes = utils.stim_start_frame (paq=paqData, stim_chan_name='maskerLED',
                                                   frame_clock=None,stim_times=None, plane=0, n_planes=1)
               shutterTimes = utils.shutter_start_frame (paq=paqData, stim_chan_name='shutterLoopback',
                                                   frame_clock=None,stim_times=None, plane=0, n_planes=1)
               # Lets organise it more for analysis friendly format
               trialStartTimes = np.unique(np.concatenate((optoStimTimes,visStimTimes),0))
               # final check if there is a problematic stim start
               first_ind = np.where(np.diff(trialStartTimes)>30) # should be at least one sec between stim starts
               first_ind = np.concatenate(([0], first_ind[0]+1))
               trialStartTimes = np.array(trialStartTimes)
               trialStartTimes = trialStartTimes[first_ind]

               trialTypes = []
               for t in trialStartTimes:
                  optoexist =  np.any(optoStimTimes== t)
                  visexist  =  np.any( visStimTimes == t)
                  if  optoexist  & visexist: 
                     trialTypes += ['Both']
                  elif optoexist &~ visexist:
                     trialTypes += ['onlyOpto']
                  elif ~optoexist & visexist:
                     trialTypes += ['onlyVis']
                  else:
                     trialTypes += ['CHECK']
               trialStartTimes = shutterTimes
               #t = [idx for idx, t_type in enumerate(trialTypes) if t_type=='Both']

            ########## Organise calcium imaging traces 
            pvals ={} 
            pvalsWilcoxon = {}
            dffTrace ={} 
            dffTrace_mean ={}
            dffTrace_median ={}
            dffAfterStim1500ms_mean ={}
            dffTrace_normalised = {}
            dffTrace_mean_normalised ={}
            
            if (info.recordingList.dffExtraction[ind] ==1):
               imData = pd.read_pickle (pathname +'imaging-data.pkl')
               fluR      = imData['flu']
               n_frames  = imData['n_frames']
               flu_raw   = imData['flu_raw']

               # Lets put nans for stimulated frames
               frameTimes = np.full((1,fluR.shape[1] ), False) # create a full false array
               for sT in shutterTimes:
                  frameTimes[:,sT:(sT+shutterLength)] = True
               fluR[:, frameTimes[0,:]] = np.nan

               # clean detrended traces
               flu = utils.clean_traces(fluR)
               flu_normalised = mfun.norm_to_zero_one (flu)

               ### Look for responsiveness for 4 trial types
               for t in tTypes:
                  if t =='All':
                     trialInd = np.transpose(list(range(len(trialStartTimes))))
                  else:
                     trialInd = [idx for idx, t_type in enumerate(trialTypes) if t_type==t]
                  
                  if len(trialInd)>1:
                     _, _, pval = utils.test_responsive  (flu, frame_clock, trialStartTimes[trialInd], pre_frames=int(np.ceil(responsiveness_test_duration/fRate)), 
                                                               post_frames=int(np.ceil(responsiveness_test_duration/fRate)), offset=simulationDur,
                                                               testType ='ttest')
                     pvals[t] = pval
                     _, _, pval = utils.test_responsive  (flu, frame_clock, trialStartTimes[trialInd], pre_frames=int(np.ceil(responsiveness_test_duration/fRate)), 
                                                               post_frames=int(np.ceil(responsiveness_test_duration/fRate)), offset=simulationDur,
                                                               testType ='wilcoxon')
                     pvalsWilcoxon[t] = pval
               nCell = len(flu)
               print(nCell)
               ### Get dff values for 4 trial types
               for indx, t in enumerate(tTypes) :
                  if t =='All':
                     trialInd = np.transpose(list(range(len(trialStartTimes))))
                  else:
                     trialInd = [idx for idx, t_type in enumerate(trialTypes) if t_type==t]
                  
                  if len(trialInd)>1:
                     dffTrace[t]      = utils.flu_splitter(flu, trialStartTimes[trialInd], pre_frames, post_frames) # Cell x time x trial
                     dffTrace_mean[t] = np.mean(dffTrace[t],2) # Cell x time
                     dffTrace_median[t] = np.median(dffTrace[t],2) # Cell x time
                     dffAfterStim1500ms_mean[t] = np.nanmean(dffTrace_mean[t][:, (pre_frames+simulationDur): (pre_frames + analysisWindowDur)],1)
                     dffTrace_normalised[t] = utils.flu_splitter(flu_normalised, trialStartTimes[trialInd], pre_frames, post_frames) # Cell x time x trial
                     dffTrace_mean_normalised[t] = np.mean (dffTrace_normalised[t],2) # Cell x time ( mean - 2)
            else:
               imData =[]

            ########## Organise pupil  traces 
            pupilTrace ={}
            pupilTrace_mean ={}
            pupilTraceVer ={}
            pupilTrace_meanVer ={}

            if (info.recordingList.pupilExtraction[ind] ==1): 
               pupilData = pd.read_pickle (pathname +'pupil-data.pkl')
               pupilrawh = pupilData['horizontalDis']
               pupilrawv = pupilData['verticalDis']
               pupilQualityh = pupilData['horizontallikelihood']
               pupilQualityv = pupilData['verticallikelihood']

               for indx, t in enumerate(tTypes) :
                  if t =='All':
                     trialInd = np.transpose(list(range(len(trialStartTimes))))
                  else:
                     trialInd = [idx for idx, t_type in enumerate(trialTypes) if t_type==t]
                  
                  if len(trialInd)>1:
                     pupilTrace[t]  = utils.trace_splitter(pupilrawh, trialStartTimes[trialInd], pre_frames, post_frames) # Cell x time x trial
                     pupilTrace_mean[t]  = np.mean(pupilTrace[t],2) # Cell x time

                     pupilTraceVer[t]  = utils.trace_splitter(pupilrawv, trialStartTimes[trialInd], pre_frames, post_frames) # Cell x time x trial
                     pupilTrace_meanVer[t]  = np.mean(pupilTraceVer[t],2) # Cell x time
            
            
            #create dff for all cells
            
            for indC, cell_dff in enumerate(flu):
               plotTrue = False
               if (len(optoStimTimes)==40): 
                  tTypes = ['Both', 'onlyOpto', 'onlyVis']
                  plotTrue =  (pvals['onlyVis'][indC] < 1e-6) or (pvals['onlyOpto'][indC] < 1e-6) or (pvals['Both'][indC] < 1e-6)
               elif (len(optoStimTimes)==20):
                  tTypes = ['Both', 'onlyVis']
                  plotTrue =  (pvals['onlyVis'][indC] < 1e-6)  or (pvals['Both'][indC] < 1e-6)

               if plotTrue:
                  fig = plt.figure(str(indC))
                  plt.switch_backend('Agg')
                  for indx, t in enumerate(tTypes):
                        pfun.lineplot_withSEM (data=dffTrace[t][indC], colorInd = indx, label=t)
                        pfun.save_figure(pathname[-20:-1]+'_DFF_mean_' + str(indC),info.figsPath +'\\cellDFF_OPN3')
                        plt.close(fig)

            #lets get recording info
            animalID = []
            x_coordinate = []
            y_coordinate = []
            z_coordinate = []
            stimuliFamilarity = []
            dataQuality =[]
            recData = []
            recID   = []
            cellID  = []

            if len(imData)>0:
               imStats = imData['stat']
               for idx, cell_flu in enumerate(flu): # from suite2p website: med: (y,x) center of cell
                  x_coordinate += [np.round(imStats[idx]['med'][1] *512/558.9) + info.recordingList['x-coordinate'] [ind]]
                  y_coordinate += [np.round(imStats[idx]['med'][0] *512/558.9) + info.recordingList['y-coordinate'] [ind]]
                  z_coordinate += [info.recordingList['z-coordinate'] [ind]]
                  animalID     += [info.recordingList['animalID'] [ind]]
                  stimuliFamilarity += [info.recordingList['stimuliFamiliarity'] [ind]]
                  dataQuality  += [info.recordingList['dataQuality'] [ind]]
                  recData  += [info.recordingList['recordingDate'] [ind]]
                  recID    += [info.recordingList['recordingID'] [ind]]
                  cellID   += [idx]
            else:
               x_coordinate = [np.nan]
               y_coordinate = [np.nan]
               z_coordinate = [np.nan]
               animalID     = [info.recordingList['animalID'] [ind]]
               stimuliFamilarity = [info.recordingList['stimuliFamiliarity'] [ind]]
               dataQuality  = [info.recordingList['dataQuality'] [ind]]
               recData  = [info.recordingList['recordingDate'] [ind]]
               recID    = [info.recordingList['recordingID'] [ind]]
               cellID   = [np.nan]

            #save outputs for population analysis
            savename = info.recordingList.analysispathname[ind] + 'extracted_variables.pkl'
            with open(savename, 'wb') as f:  # Python 3: open(..., 'wb')
               pickle.dump([trialStartTimes, trialTypes, pvals, dffTrace, 
                           dffTrace_mean, dffAfterStim1500ms_mean,
                           x_coordinate,y_coordinate,z_coordinate,
                           animalID,stimuliFamilarity,pvalsWilcoxon, dataQuality,
                           recData, recID, cellID, pupilTrace, pupilTrace_mean, 
                           pupilQualityh, pupilTraceVer, pupilTrace_meanVer, pupilQualityv,
                           dffTrace_normalised,dffTrace_mean_normalised, dffTrace_median], f)
            info.recordingList.loc[ind,'extractedVar'] = 1
         else:
            print('FAILED: no paq file : ' + pathname)
            info.recordingList.loc[ind,'extractedVar'] = 0
     # except:
      #   print('FAILED: ' + pathname)
      #   info.recordingList.loc[ind,'extractedVar'] = 0

#save info 
filenameINFO = info.analysisPath + '\\infoForAnalysis-extracted.pkl'
with open(filenameINFO, 'wb') as f:
    pickle.dump(info, f)
print('All should be done!!')

272: Creating: C:\Users\Huriye\Documents\code\clapfcstimulation\analysis\2023-08-16_2306_001\
Creating the dict for:C:\Users\Huriye\Documents\code\clapfcstimulation\analysis\2023-08-16_2306_001\
688


c:\Users\Huriye\Documents\code\clapfcstimulation\plot_funcs.py:57: RuntimeWarning: Mean of empty slice
  ylim_min = np.floor(np.nanmin(np.nanmean(data,1)))*1.1
c:\Users\Huriye\Documents\code\clapfcstimulation\plot_funcs.py:58: RuntimeWarning: Mean of empty slice
  ylim_max = np.ceil (np.nanmax(np.nanmean(data,1)))*1.1
c:\Users\Huriye\Documents\code\clapfcstimulation\plot_funcs.py:57: RuntimeWarning: Mean of empty slice
  ylim_min = np.floor(np.nanmin(np.nanmean(data,1)))*1.1
c:\Users\Huriye\Documents\code\clapfcstimulation\plot_funcs.py:58: RuntimeWarning: Mean of empty slice
  ylim_max = np.ceil (np.nanmax(np.nanmean(data,1)))*1.1
c:\Users\Huriye\Documents\code\clapfcstimulation\plot_funcs.py:57: RuntimeWarning: Mean of empty slice
  ylim_min = np.floor(np.nanmin(np.nanmean(data,1)))*1.1
c:\Users\Huriye\Documents\code\clapfcstimulation\plot_funcs.py:58: RuntimeWarning: Mean of empty slice
  ylim_max = np.ceil (np.nanmax(np.nanmean(data,1)))*1.1
c:\Users\Huriye\Documents\code\clapfcsti

NameError: name 'pupilQualityh' is not defined

In [22]:
pathname[-20:-1]+'_DFF_mean_'

'2023-08-16_2306_001_DFF_mean_'

In [ ]:
# # TEST ONE RECORNIG 
# ind = 1
# savepathname = info.recordingList.analysispathname[ind]
# filenamePAQ_analysis = [f for f in glob.glob(savepathname + '2extracted_variables.pkl')] # paq analysis file
# if (len(filenamePAQ_analysis)==1):
#     info.recordingList.loc[ind,'extractedVar'] = 1
# else:
#     print(str(ind) + ': Creating: ' + info.recordingList.analysispathname[ind])
#     #Create a huge dictionary with all cells and parameters for each cell
#     pathname = info.recordingList.analysispathname[ind]
#     print('Creating the dict for:' + pathname)

#     ########## Organise stimuli times 
#     if  (info.recordingList.paqExtraction[ind] ==1):
#         paqData = pd.read_pickle (pathname+'paq-data.pkl')
#         paqRate = paqData['rate']
#         # Get the stim start times 
#         frame_clock    = utils.paq_data (paqData, 'prairieFrame', threshold_ttl=True, plot=False)
#         optoStimTimes  = utils.paq_data (paqData, 'optoLoopback', threshold_ttl=True, plot=False)
#         if len(optoStimTimes)>500:
#             # the frame_clock is slightly longer in paq as there are some up to a sec delay from
#             # microscope to PAQI/O software.  
#             optoStimTimes = utils.stim_start_frame (paq=paqData, stim_chan_name='optoLoopback',
#                                                 frame_clock=None,stim_times=None, plane=0, n_planes=1)
#             visStimTimes = utils.stim_start_frame (paq=paqData, stim_chan_name='maskerLED',
#                                                 frame_clock=None,stim_times=None, plane=0, n_planes=1)
#             shutterTimes = utils.shutter_start_frame (paq=paqData, stim_chan_name='shutterLoopback',
#                                                 frame_clock=None,stim_times=None, plane=0, n_planes=1)

#             # Lets organise it more for analysis friendly format
#             trialStartTimes = np.unique(np.concatenate((optoStimTimes,visStimTimes),0))
#             trialTypes = []
#             for t in trialStartTimes:
#                 optoexist =  np.any(optoStimTimes== t)
#                 visexist  =  np.any( visStimTimes == t)
#                 if  optoexist  & visexist: 
#                     trialTypes += ['Both']
#                 elif optoexist &~ visexist:
#                     trialTypes += ['onlyOpto']
#                 elif ~optoexist & visexist:
#                     trialTypes += ['onlyVis']
#                 else:
#                     trialTypes += ['CHECK']
#             trialStartTimes = shutterTimes
#         #t = [idx for idx, t_type in enumerate(trialTypes) if t_type=='Both']

#         ########## Organise calcium imaging traces 
#         pvals ={} 
#         pvalsWilcoxon = {}
#         dffTrace ={} 
#         dffTrace_mean ={}
#         dffTrace_normalised ={} 
#         dffTrace_mean_normalised ={}
        
        
#         if (info.recordingList.dffExtraction[ind] ==1):
#             imData = pd.read_pickle (pathname +'imaging-data.pkl')
#             fluR      = imData['flu']
#             n_frames  = imData['n_frames']
#             flu_raw   = imData['flu_raw']

#             # Lets put nans for stimulated frames
#             frameTimes = np.full((1,fluR.shape[1] ), False) # create a full false array
#             for sT in shutterTimes:
#                 frameTimes[:,sT:(sT+shutterLength)] = True
#             fluR[:, frameTimes[0,:]] = np.nan

#             # clean detrended traces
#             flu = utils.clean_traces(fluR)
#             flu_normalised = mfun.norm_to_zero_one (flu)

#             ### Look for responsiveness for 4 trial types
#             for t in tTypes:
#                 if t =='All':
#                     trialInd = np.transpose(list(range(len(trialStartTimes))))
#                 else:
#                     trialInd = [idx for idx, t_type in enumerate(trialTypes) if t_type==t]
                
#                 if len(trialInd)>1:
#                     _, _, pval = utils.test_responsive  (flu, frame_clock, trialStartTimes[trialInd], pre_frames=int(np.ceil(responsiveness_test_duration/fRate)), 
#                                                             post_frames=int(np.ceil(responsiveness_test_duration/fRate)), offset=simulationDur,
#                                                             testType ='ttest')
#                     pvals[t] = pval
#                     _, _, pval = utils.test_responsive  (flu, frame_clock, trialStartTimes[trialInd], pre_frames=int(np.ceil(responsiveness_test_duration/fRate)), 
#                                                             post_frames=int(np.ceil(responsiveness_test_duration/fRate)), offset=simulationDur,
#                                                             testType ='wilcoxon')
#                     pvalsWilcoxon[t] = pval

#             ### Get dff values for 4 trial types
#             for indx, t in enumerate(tTypes) :
#                 if t =='All':
#                     trialInd = np.transpose(list(range(len(trialStartTimes))))
#                 else:
#                     trialInd = [idx for idx, t_type in enumerate(trialTypes) if t_type==t]
                
#                 if len(trialInd)>1:
#                     dffTrace[t]      = utils.flu_splitter(flu, trialStartTimes[trialInd], pre_frames, post_frames) # Cell x time x trial
#                     dffTrace_mean[t] = np.mean(dffTrace[t],2) # Cell x time
#                     dffTrace_normalised[t]      = utils.flu_splitter(flu_normalised, trialStartTimes[trialInd], pre_frames, post_frames) # Cell x time x trial
#                     dffTrace_mean_normalised[t] = np.mean(dffTrace_normalised[t],2) # Cell x time

#         ########## Organise pupil  traces 
#         pupilTrace = {}
#         if (info.recordingList.dffExtraction[ind] ==1): 
#             pupilData = pd.read_pickle (pathname +'pupil-data.pkl')
#             pupilraw = pupilData['verticalDis']
#             pupilraw = utils.clean_traces(pupilraw)

#             for indx, t in enumerate(tTypes) :
#                 if t =='All':
#                     trialInd = np.transpose(list(range(len(trialStartTimes))))
#                 else:
#                     trialInd = [idx for idx, t_type in enumerate(trialTypes) if t_type==t]
                
#                 if len(trialInd)>1:
#                     pupilTrace[t]  = utils.trace_splitter(pupilraw,trialStartTimes[trialInd], pre_frames, post_frames)
            
            
#             # #create dff for all cells
#             # for indC, cell_dff in enumerate(flu):
#             #    plotTrue = False
#             #    if (len(optoStimTimes)==40): 
#             #       tTypes = ['Both', 'onlyOpto', 'onlyVis']
#             #       plotTrue =  (pvals['onlyVis'][indC] < 1e-6) or (pvals['onlyOpto'][indC] < 1e-6) or (pvals['Both'][indC] < 1e-6)
#             #    elif (len(optoStimTimes)==20):
#             #       tTypes = ['Both', 'onlyVis']
#             #       plotTrue =  (pvals['onlyVis'][indC] < 1e-6)  or (pvals['Both'][indC] < 1e-6)

#             #    if plotTrue:
#             #       fig = plt.figure(str(indC))
#             #       plt.switch_backend('Agg')
#             #       for indx, t in enumerate(tTypes):
#             #             pfun.lineplot_withSEM (data=dffTrace[t][indC], colorInd = indx, label=t)
#             #             pfun.save_figure(pathname[-21:-1]+'_DFF_mean_'+str(indC),info.figsPath+'\\cellDFF\\')
#             #             plt.close(fig)

#         #lets get recording info
#         animalID = []
#         x_coordinate = []
#         y_coordinate = []
#         z_coordinate = []
#         stimuliFamilarity = []
#         dataQuality =[]
#         recData = []
#         recID   = []
#         cellID  = []
#         imStats = imData['stat']
#         for idx, cell_flu in enumerate(flu): # from suite2p website: med: (y,x) center of cell
#             x_coordinate += [np.round(imStats[idx]['med'][1] *512/558.9) + info.recordingList['x-coordinate'] [ind]]
#             y_coordinate += [np.round(imStats[idx]['med'][0] *512/558.9) + info.recordingList['y-coordinate'] [ind]]
#             z_coordinate += [info.recordingList['z-coordinate'] [ind]]
#             animalID     += [info.recordingList['animalID'] [ind]]
#             stimuliFamilarity += [info.recordingList['stimuliFamiliarity'] [ind]]
#             dataQuality  += [info.recordingList['dataQuality'] [ind]]
#             recData  += [info.recordingList['recordingDate'] [ind]]
#             recID    += [info.recordingList['recordingID'] [ind]]
#             cellID   += [idx]


#         #save outputs for population analysis
#         savename = info.recordingList.analysispathname[ind] + 'extracted_variables.pkl'
#         with open(savename, 'wb') as f:  # Python 3: open(..., 'wb')
#             pickle.dump([trialStartTimes, trialTypes, pvals, dffTrace, 
#                         dffTrace_mean, dffTrace_mean_normalised,
#                         x_coordinate,y_coordinate,z_coordinate,
#                         animalID,stimuliFamilarity,pvalsWilcoxon, dataQuality,
#                         recData, recID, cellID,pupilTrace, dffTrace_normalised], f)
#         info.recordingList.loc[ind,'extractedVar'] = 1
#     else:
#         print('FAILED: no paq file : ' + pathname)
#         info.recordingList.loc[ind,'extractedVar'] = 0

In [3]:
## Merge all datapoints & save them in a dictionary

infoPath = 'C:\\Users\\Huriye\\Documents\\code\\clapfcstimulation\\analysis\\infoForAnalysis-extracted.pkl'
info = pd.read_pickle(infoPath) 

pvalsBoth = []
pvalsVis = []
pvalsOpto = []
tTypes = ['All','Both', 'onlyOpto', 'onlyVis']

animalID = []
x_coordinate = []
y_coordinate = []
z_coordinate = []
stimuliFamilarity = []
dataQuality  = []

dff_meanBothValue  = []
dff_meanVisValue  = []
dff_meanOptoValue  = []

dff_meanBothValue_normalised  = []
dff_meanVisValue_normalised   = []
dff_meanOptoValue_normalised  = []

recData = []
recID   = []
cellID  = []
pupilID = []

pupil_resample_num = int(6*5)
empty_array_shape = (1,240)


# Create the main variables for plots by merging the extracted variables from all recordings
ty = 0
for ind, recordingDate in enumerate(info.recordingList.recordingDate):
   if (info.recordingList.extractedVar[ind] ==1):
      # load the extracted values
      pathname = info.recordingList.analysispathname[ind] 
      extData = pd.read_pickle (pathname + 'extracted_variables.pkl')
      print(ind)
      # Get Dff traces
      dff_trace = extData[24] #4 for mean, 24 for median
      if len(dff_trace)>2:
         print('putting DFF')
         if ind == 0: # For the first recording
            dff_traceBoth = dff_trace['Both']
            dff_traceVis  = dff_trace['onlyVis']
            if 'onlyOpto' in dff_trace:
               dff_traceOpto = dff_trace['onlyOpto']
            else:
               fakeOpto = np.empty (np.shape(dff_trace['Both']))
               fakeOpto[:] = np.nan
               dff_traceOpto = fakeOpto
         else:
            dff_traceBoth =  np.vstack ((dff_traceBoth , dff_trace['Both']))
            dff_traceVis  =  np.vstack ((dff_traceVis  , dff_trace['onlyVis']))
            
            if 'onlyOpto' in dff_trace:
               dff_traceOpto =  np.vstack ((dff_traceOpto , dff_trace['onlyOpto']))
            else:
               fakeOpto = np.empty (np.shape(dff_trace['Both']))
               fakeOpto[:] = np.nan
               dff_traceOpto =  np.vstack ((dff_traceOpto ,fakeOpto))
      else:
          if ind == 0:
            dff_traceVis  = np.empty (empty_array_shape)
            dff_traceBoth = np.empty (empty_array_shape)
            dff_traceOpto = np.empty (empty_array_shape)

          else:
            dff_traceVis  = np.vstack ((dff_traceVis , np.empty (empty_array_shape)))
            dff_traceBoth = np.vstack ((dff_traceBoth , np.empty (empty_array_shape)))
            dff_traceOpto = np.vstack ((dff_traceOpto , np.empty (empty_array_shape)))

      # Get Dff traces - Normalised
      dff_trace = extData[23]
      if len(dff_trace)>2:
         print('putting DFF normalised')
         if ind == 0: # For the first recording
            dff_traceBoth_normalised = dff_trace['Both']
            dff_traceVis_normalised  = dff_trace['onlyVis']
            if 'onlyOpto' in dff_trace:
               dff_traceOpto_normalised = dff_trace['onlyOpto']
            else:
               fakeOpto = np.empty (np.shape(dff_trace['Both']))
               fakeOpto[:] = np.nan
               dff_traceOpto_normalised = fakeOpto
         else:
            dff_traceBoth_normalised =  np.vstack ((dff_traceBoth_normalised , dff_trace['Both']))
            dff_traceVis_normalised  =  np.vstack ((dff_traceVis_normalised  , dff_trace['onlyVis']))
            
            if 'onlyOpto' in dff_trace:
               dff_traceOpto_normalised =  np.vstack ((dff_traceOpto_normalised , dff_trace['onlyOpto']))
            else:
               fakeOpto = np.empty (np.shape(dff_trace['Both']))
               fakeOpto[:] = np.nan
               dff_traceOpto_normalised =  np.vstack ((dff_traceOpto_normalised ,fakeOpto))
      else:
          if ind == 0:
            dff_traceVis_normalised  = np.empty (empty_array_shape)
            dff_traceBoth_normalised = np.empty (empty_array_shape)
            dff_traceOpto_normalised = np.empty (empty_array_shape)

          else:
            dff_traceVis_normalised  = np.vstack ((dff_traceVis_normalised , np.empty (empty_array_shape)))
            dff_traceBoth_normalised = np.vstack ((dff_traceBoth_normalised , np.empty (empty_array_shape)))
            dff_traceOpto_normalised = np.vstack ((dff_traceOpto_normalised , np.empty (empty_array_shape)))
      # Get pupil traces
      pupil_trace = extData[17]
      pupilQuality = extData[18]
      len_cellID   = len(extData[15])
   
      if len_cellID==0:
         len_cellID=1
      if ind == 0: # For the first recording
         if (len(pupil_trace) >0) & (pupilQuality>0.9):
            print('putting pupil 1')
            pupilID.extend([1]*len_cellID)
            pupil_traceBoth = np.tile(pupil_trace['Both'],(len_cellID,240))
            pupil_traceVis  = np.tile(pupil_trace['onlyVis'],(len_cellID,240)) 
            if 'onlyOpto' in pupil_trace:
               pupil_traceOpto = np.tile(pupil_trace['onlyOpto'],(len_cellID,240)) 
            else:
               fakeOpto = np.empty (np.tile(pupil_trace['Both'],(len_cellID,240)))
               fakeOpto[:] = np.nan
               pupil_traceOpto = fakeOpto
         else:
            pupilID.extend([0]*len_cellID)
            fake = np.empty ((len_cellID,240))
            fake[:] = np.nan
            pupil_traceBoth = fake
            pupil_traceVis  = fake
            pupil_traceOpto =  fake
      else:
         if (len(pupil_trace) >0) & (pupilQuality>0.9):
            print('putting pupil 1')
            pupilID.extend([1]*len_cellID)
            pupil_traceBoth =  np.vstack ((pupil_traceBoth, (np.tile(pupil_trace['Both'],(len_cellID,1)))))
            pupil_traceVis  =  np.vstack ((pupil_traceVis, np.tile(pupil_trace['onlyVis'],(len_cellID,1))))
            if 'onlyOpto' in pupil_trace:
               pupil_traceOpto =  np.vstack ((pupil_traceOpto,np.tile(pupil_trace['onlyOpto'],(len_cellID,1))))
            else:
               fakeOpto = np.empty (np.tile(pupil_trace['Both'],(len_cellID,1)))
               fakeOpto[:] = np.nan
               pupil_traceOpto =  np.vstack ((pupil_traceOpto ,fakeOpto))
         else:
            pupilID.extend([0]*len_cellID)
            fake = np.empty ((len_cellID,240))
            fake[:] = np.nan
            pupil_traceBoth = np.vstack ((pupil_traceBoth , fake))
            pupil_traceVis  = np.vstack ((pupil_traceVis  , fake))
            pupil_traceOpto = np.vstack ((pupil_traceOpto , fake))
         
      # Get p vals for all cells
      pvals = extData[2]
      if len(pvals)>0:
         pvalsBoth += pvals['Both'].tolist()
         pvalsVis  += pvals['onlyVis'].tolist()

         if 'onlyOpto' in dff_trace:
            pvalsOpto  += pvals['onlyOpto'].tolist()
         else:
            fakeOpto = np.empty (np.shape(pvals['Both']))
            fakeOpto[:] = 5
            pvalsOpto  += fakeOpto.tolist()
      else:
         pvalsBoth += [5]
         pvalsVis  += [5]
         pvalsOpto += [5]

      # get dff mean values for analysis window 
      dff_mean = extData[5] # mean 1500ms 
      if len(dff_mean)>0:
         print('Adding dff mean')
         print(len(dff_mean['Both'].tolist()))
         dff_meanBothValue += dff_mean['Both'].tolist()
         dff_meanVisValue  += dff_mean['onlyVis'].tolist()

         if 'onlyOpto' in dff_trace:
            dff_meanOptoValue += dff_mean['onlyOpto'].tolist()
         else:
            fakeOpto = np.empty (np.shape(dff_mean['Both']))
            fakeOpto[:]   = np.nan
            dff_meanOptoValue  += fakeOpto.tolist()
      else:
         fakeOpto = np.empty(1)
         fakeOpto[:]   = np.nan
         dff_meanBothValue += fakeOpto.tolist()
         dff_meanVisValue  += fakeOpto.tolist()
         dff_meanOptoValue += fakeOpto.tolist()


      # get the necessary info about cells 
      x_coordinate += extData[6]
      y_coordinate += extData[7]
      z_coordinate += extData[8]
      animalID     += extData[9]
      stimuliFamilarity += extData[10]
      dataQuality += extData[12]
      recData  += extData[13]
      recID    += extData[14]
      cellID   += extData[15]



0
putting DFF
putting DFF normalised
Adding dff mean
357
1
putting DFF
putting DFF normalised
Adding dff mean
376
2
putting DFF
putting DFF normalised
Adding dff mean
411
3
putting DFF
putting DFF normalised
Adding dff mean
461
4
putting DFF
putting DFF normalised
Adding dff mean
91
5
putting DFF
putting DFF normalised
Adding dff mean
12
6
putting DFF
putting DFF normalised
Adding dff mean
207
7
putting DFF
putting DFF normalised
Adding dff mean
239
8
putting DFF
putting DFF normalised
Adding dff mean
65
9
putting DFF
putting DFF normalised
Adding dff mean
235
10
putting DFF
putting DFF normalised
Adding dff mean
249
11
putting DFF
putting DFF normalised
Adding dff mean
170
12
putting DFF
putting DFF normalised
Adding dff mean
118
13
putting DFF
putting DFF normalised
Adding dff mean
418
14
putting DFF
putting DFF normalised
Adding dff mean
375
15
putting DFF
putting DFF normalised
Adding dff mean
402
16
putting DFF
putting DFF normalised
Adding dff mean
293
17
putting DFF
putting DFF 

In [4]:
# Organise & save files

# fRate = 1000/30.0
pre_frames    = 2000.0# in ms
pre_frames    = int(np.ceil(pre_frames/fRate))
post_frames   = 6000.0 # in ms
post_frames   = int(np.ceil(post_frames/fRate))
analysis_time = 1500.0 # in ms
analysis_time = int(np.ceil(analysis_time/fRate))
simulationDur_ms = 350.0 # in ms
simulationDur = int(np.ceil(simulationDur_ms/fRate))
### Lets do tiny cleaning for imaging traces
ac = np.mean(dff_traceBoth[:, 0:pre_frames],1)
ac = np.tile(ac,(np.shape(dff_traceBoth)[1],1)).transpose()
dff_traceBoth = dff_traceBoth -ac

ac = np.mean(dff_traceVis[:, 0:pre_frames],1)
ac = np.tile(ac,(np.shape(dff_traceVis)[1],1)).transpose()
dff_traceVis = dff_traceVis -ac

ac = np.mean(dff_traceOpto[:, 0:pre_frames],1)
ac = np.tile(ac,(np.shape(dff_traceOpto)[1],1)).transpose()
dff_traceOpto = dff_traceOpto -ac
                 
# ### Lets do tiny cleaning for imaging traces
# ac = np.mean(dff_traceBoth_normalised[:, 0:pre_frames],1)
# ac = np.tile(ac,(np.shape(dff_traceBoth_normalised)[1],1)).transpose()
# dff_traceBoth_normalised = dff_traceBoth_normalised -ac

# ac = np.mean(dff_traceVis_normalised[:, 0:pre_frames],1)
# ac = np.tile(ac,(np.shape(dff_traceVis_normalised)[1],1)).transpose()
# dff_traceVis_normalised = dff_traceVis_normalised -ac

# ac = np.mean(dff_traceOpto_normalised[:, 0:pre_frames],1)
# ac = np.tile(ac,(np.shape(dff_traceOpto_normalised)[1],1)).transpose()
# dff_traceOpto_normalised = dff_traceOpto_normalised -ac

### Lets do tiny cleaning for pupil traces
ac = np.mean(pupil_traceBoth[:, 0:pre_frames],1)
ac = np.tile(ac,(np.shape(pupil_traceBoth)[1],1)).transpose()
pupil_traceBoth = pupil_traceBoth -ac

ac = np.mean(pupil_traceVis[:, 0:pre_frames],1)
ac = np.tile(ac,(np.shape(pupil_traceVis)[1],1)).transpose()
pupil_traceVis = pupil_traceVis -ac

ac = np.mean(pupil_traceOpto[:, 0:pre_frames],1)
ac = np.tile(ac,(np.shape(pupil_traceOpto)[1],1)).transpose()
pupil_traceOpto = pupil_traceOpto -ac

pupil_traceOpto = resample(pupil_traceOpto, pupil_resample_num, axis=1)
pupil_traceVis = resample(pupil_traceVis, pupil_resample_num, axis=1)
pupil_traceBoth = resample(pupil_traceBoth, pupil_resample_num, axis=1)

dff_meanBoth1sec = np.nanmean(dff_traceBoth [:, pre_frames:(pre_frames + simulationDur + analysis_time)],axis=1)
dff_meanVis1sec  = np.nanmean(dff_traceVis  [:, pre_frames:(pre_frames + simulationDur + analysis_time)],axis=1)
dff_meanOpto1sec = np.nanmean(dff_traceOpto [:, pre_frames:(pre_frames + simulationDur + analysis_time)],axis=1) 

zdff_traceBoth = stats.zscore (dff_traceBoth, nan_policy='omit')
zdff_traceVis  = stats.zscore (dff_traceVis, nan_policy='omit')
zdff_traceOpto = stats.zscore (dff_traceOpto, nan_policy='omit')

filenameINFO = info.analysisPath + '\\infoForAnalysis-readyForSelectingInterestedCells.pkl'
with open(filenameINFO, 'wb') as f:
    pickle.dump((animalID, stimuliFamilarity, dataQuality,
                 recData, recID, cellID, 
                 pvalsBoth, pvalsVis, pvalsOpto,
                 dff_meanVisValue, dff_meanBothValue, 
                 dff_meanOptoValue, pupilID),f)

filenameINFO = info.analysisPath + '\\infoForAnalysis-readyForPlotting_position.pkl'
with open(filenameINFO, 'wb') as f:
    pickle.dump((x_coordinate, y_coordinate, z_coordinate ), f)
    
filenameINFO = info.analysisPath + '\\infoForAnalysis-readyForPlotting.pkl'
with open(filenameINFO, 'wb') as f:
    pickle.dump((dff_traceBoth, dff_traceVis, dff_traceOpto, 
                dff_meanBoth1sec, dff_meanVis1sec, dff_meanOpto1sec), f)

filenameINFO = info.analysisPath + '\\infoForAnalysis-readyForPlotting_normalised.pkl'
with open(filenameINFO, 'wb') as f:
    pickle.dump((dff_traceBoth_normalised, dff_traceVis_normalised, dff_traceOpto_normalised), f)

filenameINFO = info.analysisPath + '\\infoForAnalysis-readyForPlottingPupil.pkl'
with open(filenameINFO, 'wb') as f:
    pickle.dump((pupil_traceVis, pupil_traceBoth, pupil_traceOpto ), f)
print('All should be done!!')

C:\Users\Huriye\AppData\Local\Temp\ipykernel_13952\1220597084.py:55: RuntimeWarning: Mean of empty slice
  dff_meanBoth1sec = np.nanmean(dff_traceBoth [:, pre_frames:(pre_frames + simulationDur + analysis_time)],axis=1)
C:\Users\Huriye\AppData\Local\Temp\ipykernel_13952\1220597084.py:56: RuntimeWarning: Mean of empty slice
  dff_meanVis1sec  = np.nanmean(dff_traceVis  [:, pre_frames:(pre_frames + simulationDur + analysis_time)],axis=1)
C:\Users\Huriye\AppData\Local\Temp\ipykernel_13952\1220597084.py:57: RuntimeWarning: Mean of empty slice
  dff_meanOpto1sec = np.nanmean(dff_traceOpto [:, pre_frames:(pre_frames + simulationDur + analysis_time)],axis=1)


All should be done!!


In [ ]:
dff_meanBoth1sec.shape